This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L, MLP, GAT
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows  

# SEED = 0
SEED = 15
PATH = 'results/try_datasets'
SAVE = False
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:0


In [2]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'ChameleonDataset', 'CiteseerGraphDataset', 'CoraGraphDataset']
# DATASETS = ['CiteseerGraphDataset', 'CoraGraphDataset']


## Best params

In [4]:
N_RUNS = 10
ACT = nn.LeakyReLU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss() #nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'hid_dim': 16, 'leg': 'MLP'},
        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'hid_dim': 32, 'leg': 'MLP'},

        {'model': 'GAT', 'heads': 2, 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0,
          'hid_dim': 16, 'leg': 'GAT'},
          

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50, 'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': 5e-4, 'drop': .5,
         'hid_dim': 32,'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'alpha': .5, 'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'alpha': None, 'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-v2'},

        # {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},
         

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN'},
      ]


In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, exp['hid_dim'], out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=exp['drop'], norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  exp['hid_dim'], out_dim, dropout=exp['drop'])

            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': exp['drop']}
                arch = GAT(in_dim,  exp['hid_dim'], out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val1b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.568 (0.676) - 0.703 (0.730) - 0.784 (0.784) - 0.784 (0.811) - 0.649 (0.649) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in true_divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.838 (0.865) - 0.811 (0.892) - 0.838 (0.919) - 0.784 (0.919) - 0.784 (0.865) - 0.811 (0.865) - 
1: 0.622 (0.649) - 0.568 (0.595) - 0.865 (0.919) - 0.730 (0.865) - 0.595 (0.676) - 0.892 (0.946) - 0.919 (0.973) - 0.892 (0.973) - 0.973 (0.973) - 0.811 (0.865) - 0.811 (0.838) - 
2: 0.459 (0.541) - 0.514 (0.541) - 0.811 (0.811) - 0.730 (0.784) - 0.541 (0.595) - 0.838 (0.865) - 0.838 (0.919) - 0.757 (0.892) - 0.811 (0.919) - 0.730 (0.757) - 0.676 (0.703) - 
3: 0.568 (0.649) - 0.189 (0.243) - 0.784 (0.892) - 0.838 (0.865) - 0.703 (0.703) - 0.865 (0.919) - 0.811 (0.919) - 0.838 (0.919) - 0.838 (0.946) - 0.838 (0.892) - 0.703 (0.784) - 
4: 0.541 (0.568) - 0.541 (0.595) - 0.811 (0.865) - 0.784 (0.892) - 0.568 (0.649) - 0.838 (0.892) - 0.811 (0.919) - 0.865 (0.892) - 0.838 (0.919) - 0.811 (0.892) - 0.811 (0.865) - 
5: 0.595 (0.595) - 0.676 (0.757) - 0.838 (0.865) - 0.784 (0.892) - 0.568 (0.595) - 0.838 (0.892) - 0.784 (0.919) - 0.784 (0.892) - 0.811 (0.865) - 0.892 (0.892) - 0.811 (0.865) - 
6: 

In [ ]:
if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_acc_val1.to_csv(PATH + 'best_params_val_test_' + timestr)
    table_acc_val1.to_csv(PATH + 'best_params_best_val_' + timestr)

In [ ]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.586486,0.541176,0.451351,0.608333,0.5811,0.5058
Kipf-A,0.554054,0.539216,0.402703,0.345614,0.6778,0.7769
MLP,0.767568,0.823529,0.718919,0.500439,0.5693,0.5749
GAT,0.578378,0.550980,0.397297,0.602632,0.6357,0.6134
A-GCN-normA,0.845946,0.849020,0.794595,0.469737,0.7022,0.8085
A-GCN,0.810811,0.864706,0.778378,0.447149,0.6698,0.7587
A-GCN-v2,0.840541,0.876471,0.762162,0.453509,0.6740,0.7738
H-GCN-normH,0.845946,0.858824,0.772973,0.423465,0.5619,0.7357
H-GCN,0.848649,0.862745,0.789189,0.443860,0.6710,0.7593
W-GCN-normA,0.818919,0.849020,0.718919,0.418860,0.2202,0.1749


In [ ]:
table_acc_val1b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.583784,0.556863,0.445946,0.613816,0.5794,0.5045
Kipf-A,0.556757,0.539216,0.402703,0.343640,0.6769,0.7732
MLP,0.772973,0.821569,0.718919,0.510307,0.5672,0.5704
GAT,0.583784,0.533333,0.413514,0.601974,0.6556,0.6444
A-GCN-normA,0.843243,0.852941,0.786486,0.471053,0.7002,0.8077
A-GCN,0.818919,0.878431,0.778378,0.447807,0.6668,0.7609
A-GCN-v2,0.827027,0.872549,0.778378,0.452632,0.6775,0.7690
H-GCN-normH,0.845946,0.839216,0.748649,0.427412,0.5703,0.7350
H-GCN,0.848649,0.860784,0.772973,0.442544,0.6699,0.7560
W-GCN-normA,0.808108,0.845098,0.735135,0.430044,0.2357,0.3070


In [ ]:
table_acc1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.637838,0.611765,0.537838,0.626974,0.5848,0.5090
Kipf-A,0.578378,0.562745,0.432432,0.351535,0.6867,0.7869
MLP,0.813514,0.858824,0.759459,0.519737,0.5750,0.5774
GAT,0.632432,0.600000,0.497297,0.619737,0.6655,0.6492
A-GCN-normA,0.905405,0.909804,0.843243,0.497149,0.7104,0.8130
A-GCN,0.908108,0.913725,0.862162,0.468860,0.6771,0.7682
A-GCN-v2,0.910811,0.915686,0.862162,0.487281,0.6871,0.7858
H-GCN-normH,0.916216,0.894118,0.827027,0.441667,0.5788,0.7423
H-GCN,0.910811,0.917647,0.859459,0.460746,0.6753,0.7674
W-GCN-normA,0.864865,0.878431,0.794595,0.441009,0.2601,0.3085


## Initial Params

In [ ]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 10
N_EPOCHS = 200  # 500
LR = .01
WD = 5e-4
DROPOUT = .5

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 16

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'MLP', 'leg': 'MLP-conf1'},
        {'model': 'GAT', 'heads': 2, 'hid_dim': 16, 'leg': 'GAT'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  HID_DIM, out_dim, dropout=DROPOUT, act=ACT, last_act=LAST_ACT)
            
            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': DROPOUT}
                arch = GAT(in_dim,  HID_DIM, out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_acc_val2.to_csv(PATH + 'init_params_val_test_' + timestr)
    table_acc_val2.to_csv(PATH + 'init_params_best_val_' + timestr)

TexasDataset
0: 0.649 (0.649) - 0.676 (0.730) - 0.784 (0.784) - 0.622 (0.649) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in true_divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.351 (0.514) - 0.541 (0.541) - 0.568 (0.649) - 0.568 (0.595) - 0.811 (0.838) - 0.784 (0.784) - 
1: 0.568 (0.595) - 0.676 (0.703) - 0.757 (0.757) - 0.595 (0.622) - 0.486 (0.649) - 0.622 (0.622) - 0.541 (0.622) - 0.595 (0.622) - 0.649 (0.757) - 0.649 (0.676) - 
2: 0.514 (0.541) - 0.459 (0.514) - 0.730 (0.757) - 0.514 (0.514) - 0.514 (0.595) - 0.514 (0.541) - 0.270 (0.486) - 0.514 (0.541) - 0.649 (0.730) - 0.378 (0.541) - 
3: 0.595 (0.649) - 0.622 (0.622) - 0.838 (0.865) - 0.486 (0.649) - 0.595 (0.622) - 0.541 (0.568) - 0.514 (0.514) - 0.486 (0.595) - 0.811 (0.865) - 0.595 (0.703) - 
4: 0.514 (0.568) - 0.568 (0.568) - 0.811 (0.865) - 0.568 (0.595) - 0.432 (0.541) - 0.486 (0.595) - 0.514 (0.568) - 0.486 (0.514) - 0.649 (0.730) - 0.514 (0.622) - 
5: 0.568 (0.622) - 0.514 (0.595) - 0.730 (0.865) - 0.568 (0.649) - 0.514 (0.595) - 0.568 (0.568) - 0.514 (0.622) - 0.622 (0.676) - 0.703 (0.865) - 0.622 (0.676) - 
6: 0.541 (0.541) - 0.568 (0.568) - 0.649 (0.703) - 0.541 (0.568) - 0.568 (0.595) - 

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.583784,0.531373,0.427027,0.629825,0.7145,0.8117
Kipf-A,0.591892,0.445098,0.370270,0.270175,0.6624,0.7683
MLP-conf1,0.751351,0.829412,0.721622,0.469737,0.5654,0.5760
GAT,0.562162,0.541176,0.440541,0.624781,0.7142,0.8086
A-GCN-normA,0.494595,0.482353,0.421622,0.377632,0.6363,0.7846
A-GCN,0.521622,0.425490,0.348649,0.217982,0.4838,0.6267
H-GCN-normH,0.443243,0.470588,0.335135,0.327412,0.7042,0.8129
GCN-H,0.554054,0.466667,0.354054,0.227851,0.4854,0.5788
W-GCN-normA,0.710811,0.700000,0.618919,0.485526,0.7086,0.8178
W-GCN,0.605405,0.566667,0.518919,0.228070,0.4986,0.5458


In [ ]:
table_acc_val2b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.586486,0.515686,0.440541,0.638377,0.7117,0.8087
Kipf-A,0.600000,0.500000,0.429730,0.319298,0.6665,0.7691
MLP-conf1,0.754054,0.837255,0.735135,0.485746,0.5626,0.5697
GAT,0.570270,0.511765,0.427027,0.645395,0.7078,0.8121
A-GCN-normA,0.556757,0.490196,0.472973,0.449123,0.6754,0.7977
A-GCN,0.510811,0.450980,0.405405,0.247368,0.5026,0.6278
H-GCN-normH,0.527027,0.480392,0.402703,0.333991,0.7030,0.8136
GCN-H,0.537838,0.456863,0.402703,0.257456,0.5052,0.6163
W-GCN-normA,0.748649,0.760784,0.686486,0.511404,0.7058,0.8173
W-GCN,0.602703,0.592157,0.610811,0.277412,0.5625,0.6663


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.613514,0.592157,0.527027,0.651535,0.7192,0.8185
Kipf-A,0.621622,0.527451,0.454054,0.326535,0.6733,0.7802
MLP-conf1,0.794595,0.858824,0.767568,0.503947,0.5689,0.5804
GAT,0.613514,0.605882,0.500000,0.660526,0.7253,0.8218
A-GCN-normA,0.605405,0.554902,0.529730,0.459430,0.6799,0.8068
A-GCN,0.583784,0.492157,0.451351,0.267105,0.5117,0.6375
H-GCN-normH,0.575676,0.550980,0.462162,0.350000,0.7151,0.8213
GCN-H,0.591892,0.523529,0.467568,0.264035,0.5146,0.6269
W-GCN-normA,0.789189,0.788235,0.727027,0.524123,0.7176,0.8250
W-GCN,0.678378,0.652941,0.651351,0.287939,0.5713,0.6728
